#Format Data for Experiment in ABM environment

In [1]:
#Import Packages
#Needed for moving to output
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as pl
import itertools
from collections import Counter

In [2]:
#Import other needed packages
from vivarium.library.units import units, remove_units
from tumor_tcell.experiments.main import tumor_tcell_abm
from tumor_tcell.experiments.main import plots_suite
from tumor_tcell.experiments.main import get_tcells
from tumor_tcell.experiments.main import get_tumors
from tumor_tcell.library.phylogeny import get_phylogeny

Loading chipmunk for Linux (64bit) [/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/vivarium_env/lib/python3.8/site-packages/pymunk/libchipmunk.so]


In [3]:
#Read in the data from parent directory
analysis_dir = '/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/out/analysis/'
experiment_id = 'Tcell/'
experiment_dir = analysis_dir+experiment_id
os.chdir(experiment_dir)
os.getcwd()

'/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/out/analysis/Tcell'

In [4]:
#Read in the data from parent directory
df_Tcell_exp = pd.read_csv('Tcell_exp_1of4.csv')
df_Tcell_exp.rename(columns = {"Unnamed: 0": "cell_index"},  inplace = True) 
df_Tcell_exp.set_index('cell_index', inplace = True)
df_Tcell_exp

,AbsoluteY,AbsoluteX,H2Db,H2Kb,PD1,PDL1,Region,Cell Type,IFNg
cell_index,,,,,,,,,
77553,155.454832,9.276161,9547.861827,4250.662263,7408.539602,11613.846635,Tcelld3,PDL1n,0
77554,113.548926,93.777729,8863.661262,20917.988305,5212.594734,32846.711076,Tcelld3,PDL1n,0
77555,113.889961,36.642164,4865.223521,10164.673741,3492.627893,22678.359845,Tcelld3,PDL1n,0
77556,145.110687,34.251813,4733.793343,9145.564329,4644.524796,15088.846405,Tcelld3,PDL1n,0
77557,85.856085,98.566209,3729.474159,15796.171936,3646.544442,22717.793153,Tcelld3,PDL1n,0
...,...,...,...,...,...,...,...,...,...
87304,151.809317,312.096487,10730.696024,27264.603473,13662.254926,41131.198511,Tcelld3,PD1n,0
87305,48.249875,352.214634,7230.250008,13097.754667,8677.908784,27377.064199,Tcelld3,PD1n,0
87306,92.605867,163.026306,19642.008088,46297.903400,12657.824880,40202.083794,Tcelld3,PD1n,0


In [5]:
#Set bounds
bounds = [1800,1800]

#Create a copy
df_Tcell_exp1 = df_Tcell_exp.copy()

#Column Number for iloc iterating
Ypos = 0
Xpos = 1
H2Kb = 3
PD1 = 4
PDL1 = 5
Celltype = 7
IFNg_con = 8 

tumor_diameter = 15
tcell_diameter = 7.5

tumors = {}
IFNg = {}
tcells = {}

for i in range(len(df_Tcell_exp1)):
  if df_Tcell_exp1.iloc[i, Celltype] == 'PDL1p' or df_Tcell_exp1.iloc[i, Celltype] == 'PDL1n':
    tumors.update({
        'tumor_'+ str(df_Tcell_exp1.index[i]) : {
            'location': [df_Tcell_exp1.iloc[i, Xpos]*units.um,df_Tcell_exp1.iloc[i, Ypos]*units.um],
            'type': 'tumor',
            'cell_state': df_Tcell_exp1.iloc[i, Celltype],
            'PDL1': df_Tcell_exp1.iloc[i, PDL1],
            'MHCI': df_Tcell_exp1.iloc[i, H2Kb],
            'diameter': tumor_diameter*units.um,
        }
    })
    
    IFNg.update({
        df_Tcell_exp1.index[i] : {
            'amount': df_Tcell_exp1.iloc[i, IFNg_con],
            'location': [df_Tcell_exp1.iloc[i, Xpos]*units.um,df_Tcell_exp1.iloc[i, Ypos]*units.um],
        }
    })

  else: 
    tcells.update({
        'tcell_'+ str(df_Tcell_exp1.index[i]) : {
            'location': [df_Tcell_exp1.iloc[i, Xpos]*units.um,df_Tcell_exp1.iloc[i, Ypos]*units.um],
            'type': 'tcell',
            'cell_state': df_Tcell_exp1.iloc[i, Celltype],
            'PD1': df_Tcell_exp1.iloc[i, PD1],
            'diameter': tcell_diameter*units.um,
        }
    })
    IFNg.update({
        df_Tcell_exp1.index[i]: {
        'amount': df_Tcell_exp1.iloc[i, IFNg_con],
        'location': [df_Tcell_exp1.iloc[i, Xpos]*units.um,df_Tcell_exp1.iloc[i, Ypos]*units.um],
        }
    })

tcells

{'tcell_87252': {'location': [95.74551977539062 <Unit('micrometer')>,
   70.05515087890626 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 27908.99231994004,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_87253': {'location': [93.95918774414065 <Unit('micrometer')>,
   73.98576245117188 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 50000.0,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_87254': {'location': [71.84154760742189 <Unit('micrometer')>,
   175.17935968017574 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 50000.0,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_87255': {'location': [89.45419421386721 <Unit('micrometer')>,
   172.37825244140626 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 30515.988131702165,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_87256': {'location': [90.07165136718748 <Unit('micrometer')>,
   166.85268511962892 <Unit('micrometer')>]

#Run tumor-tcell ABM Experiment

In [ ]:
#using experimental CODEX data
data = tumor_tcell_abm(tumors=tumors, 
                       tcells=tcells, 
                       total_time=259200, 
                       bounds=[b*units.um for b in bounds], 
                       halt_threshold=10000,
                       emit_step=600,
                       emitter='database',)
data;

Initializing experiment tumor_tcell_20210716.142722


 92%|█████████▏| 398/432 [68:53:48<18:49:22, 1993.03s/it]